Dynamic programming works by breaking down complex problems into simpler subproblems. Then, finding optimal solutions to these subproblems. Memorization is a method that saves the outcomes of these processes so that the corresponding answers do not need to be computed when they are later needed. Saving solutions save time on the computation of subproblems that have already been encountered. 

In [ ]:
# 时间复杂度：O(n)
# 空间复杂度：O(n)
class Solution:
    def fib(self, n: int) -> int:
       
        # 排除 Corner Case
        if n == 0:
            return 0
        
        # 创建 dp table 
        dp = [0] * (n + 1)

        # 初始化 dp 数组
        dp[0] = 0
        dp[1] = 1

        # 遍历顺序: 由前向后。因为后面要用到前面的状态
        for i in range(2, n + 1):

            # 确定递归公式/状态转移公式
            dp[i] = dp[i - 1] + dp[i - 2]
        
        # 返回答案
        return dp[n]


# 时间复杂度：O(n)
# 空间复杂度：O(1)
class Solution:
    def fib(self, n: int) -> int:
        if n <= 1:
            return n
        
        prev1, prev2 = 0, 1
        
        for _ in range(2, n + 1):
            curr = prev1 + prev2
            prev1, prev2 = prev2, curr
        
        return prev2

例如：有N件物品和一个最多能背重量为W 的背包。第i件物品的重量是weight[i]，得到的价值是value[i] 。每件物品只能用一次，求解将哪些物品装入背包里物品价值总和最大。

dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - weight[i]] + value[i])

i 来表示物品、j表示背包容量。
即dp[i][j] 表示从下标为[0-i]的物品里任意取，放进容量为j的背包，价值总和最大是多少。

<img src="pic/0-1-Knapsack-660.webp" width="50%">

## 二维数组

In [ ]:
## 二维数组

# 物品重量 [w1, w2, ...], 价值 [v1, v2, ...], 背包容量 W
n = len(weights)
dp = [[0] * (W + 1) for _ in range(n + 1)]

# *** 方式1：外层物品，内层容量（标准写法）
for i in range(n):              # Items 0..n-1
    for j in range(W + 1):      # Capacities 0..W
        if j >= weights[i]:     # Direct access to weights[i]
            dp[i+1][j] = max(dp[i][j], dp[i][j-weights[i]] + values[i])
        else:
            dp[i+1][j] = dp[i][j]
            
# 方式2：外层容量，内层物品（顺序可互换）
for j in range(W + 1):      # Capacity from 0 to W
    for i in range(n):      # Items from 0 to n-1
        if weights[i] <= j:
            dp[i+1][j] = max(dp[i][j], dp[i][j - weights[i]] + values[i])
        else:
            dp[i+1][j] = dp[i][j]
# 结果相同，因为 dp[i][j] 只依赖 dp[i-1][...]，顺序不影响。

## 一维数组
### 01背包

对于背包问题其实状态都是可以压缩的。

在使用二维数组的时候，递推公式：dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - weight[i]] + value[i]);

动态规划中dp[j]是由dp[j-weight[i]]推导出来的，然后取max(dp[j], dp[j - weight[i]] + value[i])

In [ ]:
## 一维数组
### 01背包
# 0/1 Knapsack (Each item can be used at most once)

# *** 正确写法（01背包）
dp = [0] * (W + 1)

for i in range(n):
    for j in range(W, weights[i] - 1, -1):  # 逆序
        dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 为什么正确？
    # 外层循环物品，内层逆序容量：
    # 每个物品 i 只会被处理一次。
    # 由于容量是 逆序更新，dp[j - weights[i]] 总是来自 上一轮（未包含当前物品） 的值，确保不会重复选取。


# 错误写法（正序）：
for i in range(n):
    for j in range(weights[i], W + 1):  # 正序
        dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 问题：dp[j - weights[i]] 可能已经被当前物品更新过，导致重复选取。
# Why reverse order?
    # If we go left-to-right, dp[j - w[i]] may have already been updated in the current iteration, leading to multiple picks of the same item.
    # Example (if we go left-to-right):
    # For i=0 (item 1, w=1, v=15):
    # dp[1] = max(dp[1], dp[0] + 15) = 15
    # dp[2] = max(dp[2], dp[1] + 15) = 30 (item 1 picked twice!)
    # This is wrong for 0/1 Knapsack (but correct for Unbounded Knapsack).


# 错误写法（交换顺序）Incorrect Order：
for j in range(W + 1):  # 外层容量 Outer: capacity
    for i in range(n):  # 内层物品 Inner: items
        if weights[i] <= j:
            dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 问题：dp[j] 会多次更新，导致同一物品被选多次（变成完全背包）。
# What happens?
    # When j=2:
    #     dp[2] = max(dp[2], dp[0] + 20) = 20 (pick item 2)
    # When j=4:
    #     dp[4] = max(dp[4], dp[2] + 20) = 40 (item 2 picked again!)
    # This violates the 0/1 constraint (each item can be used at most once).
# 为什么错误？
    # 问题1：同一物品可能在 同一轮容量更新中被多次选取。
    # 问题2：dp[j - weights[i]] 可能已经被 当前物品更新过，导致重复计算。



# 对于任意物品 i 和容量 j：
# 逆序更新：
# dp[j] 只依赖 dp[j - w[i]]（上一轮的值）。
# 因此，物品 i 不会被重复加入。
# 正序更新：
# dp[j] 可能依赖已经被当前物品更新过的 dp[j - w[i]]。
# 导致物品 i 被多次选取（变成完全背包问题）。

### 完全背包

In [ ]:
### 完全背包
# Unbounded Knapsack (Items can be reused)

# *** 正确写法1（外层物品，内层容量正序）1D DP - Must Forward Iterate
dp = [0] * (W + 1)

for i in range(n):
    for j in range(weights[i], W + 1):  # 正序
        dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# Why forward order?
    # We want to reuse items, so dp[j - w[i]] should include the current item if possible.
    # Example (for i=0, item 1, w=1, v=15):
    # dp[1] = max(0, dp[0] + 15) = 15
    # dp[2] = max(0, dp[1] + 15) = 30 (item 1 used twice)
    # dp[3] = 45, ..., dp[5] = 75 (item 1 used five times).

# 正确写法2（外层容量，内层物品）：
for j in range(W + 1):
    for i in range(n):
        if weights[i] <= j:
            dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 结果相同（求最大价值时）。

# Wrong Approach (1D DP - Reverse Order)
dp = [0] * (W + 1)

for i in range(n):
    for j in range(W, w[i] - 1, -1):  # Reverse order (like 0/1 Knapsack)
        dp[j] = max(dp[j], dp[j - w[i]] + v[i])

print(dp[W])  # Output: 65 (Same as 0/1 Knapsack - Wrong for Unbounded!)
# What happens?
    # Reverse order prevents reusing items (like 0/1 Knapsack).
    # We get the 0/1 Knapsack solution (65) instead of the unbounded solution (75).

In [ ]:
322. Coin Change

## (1) 组合数（顺序不重要）

二维DP数组递推公式： dp[i][j] = dp[i - 1][j] + dp[i - 1][j - nums[i]];
去掉维度i 之后，递推公式：dp[j] = dp[j] + dp[j - nums[i]] ，即：dp[j] += dp[j - nums[i]]

In [ ]:
# 3. 组合数 vs 排列数（完全背包变种）
# (1) 组合数（顺序不重要）
# 问题：w = [1, 2, 3]，W = 4，求有多少种组合方式？
dp = [0] * (W + 1) # dp[j] 表示凑出重量 j 的组合数
dp[0] = 1  # 初始化：凑出重量 0 的组合数为 1（即不选任何物品）
# dp = [1, 0, 0, 0, 0]（W=4，所以数组长度为 5）。


# 外层物品，内层容量（组合数）
for i in range(n):
    # 对当前物品 weights[i]，从 j = weights[i] 开始更新到 W。
    # 正序更新：确保可以重复使用当前物品（但组合数问题中顺序不重要）。
    for j in range(weights[i], W + 1):
        dp[j] += dp[j - weights[i]]
# 逻辑：如果当前物品 weights[i] 可以放入容量 j，则组合数增加 dp[j - weights[i]]。
# 物理意义：
    # dp[j - weights[i]] 是 不选当前物品时 凑出 j - weights[i] 的组合数。
    # 现在选了当前物品，组合数自然累加。
    # dp[j] += ...：如果选了当前物品 weights[i]，那么凑出 j 的组合数 增加 dp[j - weights[i]] 种方式。

# 为什么外层循环是物品，内层循环是容量？
    # 外层循环物品：确保 组合的唯一性（顺序不重要）。
    # 例如，[1, 2] 和 [2, 1] 不会被重复计算，因为 2 是在 1 之后处理的。
    # 内层循环容量（正序）：允许 重复使用当前物品（完全背包特性）。

# 为什么能保证顺序不重要？
    # 外层按物品顺序处理，固定了物品的选择顺序。
    # 例如，在处理完物品 1 后，再处理物品 2，因此 [1,2] 会被计算，但 [2,1] 不会重复出现。

print(dp[W])  # 输出：4（[1,1,1,1], [1,1,2], [1,3], [2,2]）
# 说明：[1,1,2] 和 [2,1,1] 算作同一种组合。

# Step 1: 处理物品 1（重量 1）
# for j in range(1, 5):  # j = 1, 2, 3, 4
#     dp[j] += dp[j - 1]
# j=1：dp[1] += dp[0] → dp = [1, 1, 0, 0, 0]（组合 [1]）
# j=2：dp[2] += dp[1] → dp = [1, 1, 1, 0, 0]（组合 [1,1]）
# j=3：dp[3] += dp[2] → dp = [1, 1, 1, 1, 0]（组合 [1,1,1]）
# j=4：dp[4] += dp[3] → dp = [1, 1, 1, 1, 1]（组合 [1,1,1,1]）  ***

# Step 2: 处理物品 2（重量 2）
# for j in range(2, 5):  # j = 2, 3, 4
#     dp[j] += dp[j - 2]
# j=2：dp[2] += dp[0] → dp = [1, 1, 2, 1, 1]（新增组合 [2]）
# j=3：dp[3] += dp[1] → dp = [1, 1, 2, 2, 1]（新增组合 [1,2]）
# j=4：dp[4] += dp[2] → dp = [1, 1, 2, 2, 3]（新增组合 [2,2] 和 [1,1,2]） ***

# Step 3: 处理物品 3（重量 3）
# for j in range(3, 5):  # j = 3, 4
#     dp[j] += dp[j - 3]
# j=3：dp[3] += dp[0] → dp = [1, 1, 2, 3, 3]（新增组合 [3]）
# j=4：dp[4] += dp[1] → dp = [1, 1, 2, 3, 4]（新增组合 [1,3]） ***
# 最终结果
# dp = [1, 1, 2, 3, 4]  # dp[4] = 4
# 组合方式：
# [1,1,1,1]
# [1,1,2]
# [2,2]
# [1,3]

# 具体例子
# 假设 weights = [1, 2, 3]，W = 4，我们来看如何计算 dp[4]：
# (1) 处理物品 1（重量 1）
# dp[4] += dp[4 - 1]（即 dp[4] += dp[3]）
# dp[3] 表示凑出 3 的组合数，现在加上 1 就能凑出 4。
# 例如：如果 dp[3] 包含 [1,1,1] 和 [3]，那么加上 1 后变成 [1,1,1,1] 和 [3,1]。

# (2) 处理物品 2（重量 2）
# dp[4] += dp[4 - 2]（即 dp[4] += dp[2]）
# dp[2] 表示凑出 2 的组合数（如 [1,1] 和 [2]），加上 2 后变成 [1,1,2] 和 [2,2]。

# (3) 处理物品 3（重量 3）
# dp[4] += dp[4 - 3]（即 dp[4] += dp[1]）
# dp[1] 表示凑出 1 的组合数（如 [1]），加上 3 后变成 [1,3]。

In [ ]:
518. Coin Change II

## (2) 排列数（顺序重要）

In [ ]:
# (2) 排列数（顺序重要）
# 问题：w = [1, 2, 3]，W = 4，求有多少种排列方式？
dp = [0] * (W + 1)
dp[0] = 1  # 初始化

# 外层容量，内层物品（排列数）
for j in range(W + 1):
    for i in range(n):
        if w[i] <= j:
            dp[j] += dp[j - w[i]]

print(dp[W])  # 输出：7（[1,1,1,1], [1,1,2], [1,2,1], [2,1,1], [2,2], [1,3], [3,1]）
# 说明：[1,1,2] 和 [2,1,1] 算作不同排列。

# 具体例子分析（w = [1, 2], W = 3）
# 目标：凑出总和为3的所有方式（数字可重复使用）。
# (1) 排列数代码的执行过程
# 初始化：dp = [1, 0, 0, 0]

# 1. 当 j = 1（和为1）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 1）：
    # dp[1] += dp[0] → dp = [1, 1, 0, 0, 0]
    # 新增排列：[1]
    
    # 物品 2（2 > 1）、物品 3（3 > 1）：跳过

# 2. 当 j = 2（和为2）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 2）：
    # dp[2] += dp[1] → dp = [1, 1, 1, 0, 0]
    # 新增排列：[1,1]
    
    # 物品 2（2 ≤ 2）：
    # dp[2] += dp[0] → dp = [1, 1, 2, 0, 0]
    # 新增排列：[2]
    
    # 物品 3（3 > 2）：跳过

# 3. 当 j = 3（和为3）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 3）：
    # dp[3] += dp[2] → dp = [1, 1, 2, 2, 0]
    # 新增排列：[1,1,1] 和 [2,1]
    
    # 物品 2（2 ≤ 3）：
    # dp[3] += dp[1] → dp = [1, 1, 2, 3, 0]
    # 新增排列：[1,2]
    
    # 物品 3（3 ≤ 3）：
    # dp[3] += dp[0] → dp = [1, 1, 2, 4, 0]
    # 新增排列：[3]

# 4. 当 j = 4（和为4）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 4）：
    # dp[4] += dp[3] → dp = [1, 1, 2, 4, 4]
    # 新增排列：[1,1,1,1]、[2,1,1]、[1,2,1]、[3,1] ***
    
    # 物品 2（2 ≤ 4）：
    # dp[4] += dp[2] → dp = [1, 1, 2, 4, 6]
    # 新增排列：[1,1,2]、[2,2] ***
    
    # 物品 3（3 ≤ 4）：
    # dp[4] += dp[1] → dp = [1, 1, 2, 4, 7]
    # 新增排列：[1,3] ***

377. Combination Sum IV

In [ ]:
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        if len(nums) <= 1:
            return len(nums)
        dp = [1] * len(nums)
        result = 0
        for i in range(1, len(nums)):
            for j in range(0, i):
                if nums[i] > nums[j]:
                    dp[i] = max(dp[i], dp[j] + 1)
            result = max(result, dp[i]) #取长的子序列
        return result